# Dataset

In [ ]:
!pip install eurostat

In [ ]:
import pandas as pd
import eurostat
df = eurostat.get_data_df("ei_bsco_m")
df

In [ ]:
df[["indic", "s_adj", "unit", "geo\\time"]].describe()

In [ ]:
df["indic"].unique()

Spalten aufräumen

In [ ]:
df = df.rename(columns={"geo\\time": "country"})

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f.split("M")[0] + "-" + f.split("M")[1] + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

In [ ]:
realnames_i = eurostat.get_dic('indic')
[f'{i}: {realnames_i[i]}' for i in df["indic"].unique()]

# Balkendiagramm und Histogramm

## Barchart

In [ ]:
de_cci = df[(df["country"] == "DE") & (df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
de_cci = de_cci[[c for c in de_cci.columns if isinstance(c, datetime) and (c.year==2020 or c.year==2019)]]
de_cci.index = ["Consumer confidence indicator"]
de_cci.transpose()[::-1].plot.bar(figsize=(16,9))

In [ ]:
cci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
cci = cci[["country", datetime(2020,10,1)]].set_index("country")
cci

In [ ]:
cci.plot.bar(figsize=(16,9))

In [ ]:
realnames_g = eurostat.get_dic('geo')
realnames_g['DE'] = "Germany"
realnames_g['EA19'] = "EURO"
realnames_g['EU27_2020'] = "EU"

In [ ]:
cci.index = [realnames_g[i] for i in cci.index]
cci.sort_values(datetime(2020, 10, 1)).plot.barh(figsize=(9, 9))

## Histogramm

In [ ]:
cci.plot.hist()

In [ ]:
cci.plot.hist(bins=20)

# Fieberkurven

# Boxplots

## 10 Jahre

In [ ]:
csmci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
csmci

In [ ]:
columns = [datetime(y, m, 1) for y in range(2010, 2021) for m in range(1, 13)]
# die letzten beiden Spalten sind für November/Dezember 2020, da gibt es heute noch keine Daten
csmci_unwrapped = pd.melt(csmci[["country"] + columns[:-2]].dropna(), 
                          id_vars=["country"], var_name="month", value_name="csmci")
csmci_unwrapped

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
csmci_unwrapped["country"] = [realnames_g[i] for i in csmci_unwrapped["country"]]
labels = csmci_unwrapped.groupby("country").agg({"csmci": "median"}).sort_values("csmci").index.values
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

## 20 Jahre

In [ ]:
csmci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
columns = [datetime(y, m, 1) for y in range(2000, 2021) for m in range(1, 13)]
# die letzten beiden Spalten sind für November/Dezember 2020, da gibt es heute noch keine Daten
csmci_unwrapped = pd.melt(csmci[["country"] + columns[:-2]].dropna(), 
                          id_vars=["country"], var_name="month", value_name="csmci")

In [ ]:
csmci_unwrapped["country"] = [realnames_g[i] for i in csmci_unwrapped["country"]]

In [ ]:
labels = csmci_unwrapped.groupby("country").agg({"csmci": "median"}).sort_values("csmci").index.values

In [ ]:
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

In [ ]:
plt.figure(figsize=(8, 16))
sns.violinplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

In [ ]:
sns.violinplot(y="country", x="csmci", 
               data=csmci_unwrapped[csmci_unwrapped["country"].isin(["Greece", "Germany", "Sweden", "Denmark"])],
               order=["Greece", "Germany", "Sweden", "Denmark"], palette="viridis")

# Zeitreihen

In [ ]:
# Länder wählen
cci_mix = df[(df["country"].isin(["DE", "EA19", "SE"])) & 
             (df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
# nur bestimmte Spalte behalten und transponieren (Zeit nach unten)
cci_mix = cci_mix[[c for c in cci_mix.columns 
                     if (isinstance(c, datetime) and c.year>=2000) or c == "country"]].set_index("country").transpose()
# Index in Zeit wandeln
cci_mix.index = pd.DatetimeIndex(cci_mix.index)

In [ ]:
cci_mix.plot()

In [ ]:
cci_mix.resample("Q").mean().plot()

# Korrelationen der Indikatoren miteinander vergleichen

In [ ]:
import scipy.stats as stats
indicators = df["indic"].unique()
corr = []
for i1 in indicators:
    res = []
    d1 = df[(df["indic"] == i1) & (df["s_adj"] == "NSA") & (df["country"] == "DE")].transpose()
    for i2 in indicators:
        d2 = df[(df["indic"] == i2) & (df["s_adj"] == "NSA") & (df["country"] == "DE")].transpose()
        r, p = stats.pearsonr(d1[[isinstance(d, datetime) and d.year>=2000 for d in d1.index]].iloc[:,0].values, 
                              d2[[isinstance(d, datetime) and d.year>=2000 for d in d2.index]].iloc[:,0].values)
        res.append(r)
    corr.append(res)

In [ ]:
y20 = [datetime(y, m, 1) for y in range(2000, 2021) for m in range(1, 13)][:-2]
de20 = df[(df["s_adj"] == "NSA") & (df["country"] == "DE")].set_index("indic")[y20].transpose()
de20.index = pd.DatetimeIndex(de20.index)

In [ ]:
de20

In [ ]:
import scipy.stats as stats
corr = []
indicators = de20.columns
# Korrelationen berechnen
for i1 in indicators:
    res = []
    for i2 in indicators:
        r, p = stats.pearsonr(de20[i1].values, de20[i2].values)
        res.append(r)
    corr.append(res)

# in Dataframe mit richtigen Spalte und Zeilen wandeln
real_indicators = [realnames_i[i] for i in indicators]
ihm = pd.DataFrame(corr, index=real_indicators, columns=real_indicators)
ihm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,12))
sns.heatmap(ihm, cmap="viridis", vmin=-1, vmax=1)

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(ihm, cmap="RdBu", vmin=-1, vmax=1)

In [ ]:
ihm.to_excel('ihm.xlsx')

In [ ]:
de = df[(df["country"] == "DE") & (df["s_adj"] == "NSA")]
de

In [ ]:
de20.plot.scatter(x="BS-SFSH", y="BS-CSMCI")

In [ ]:
import seaborn as sns
import scipy.stats as stats
sns.jointplot(x=de20["BS-SFSH"], y=de20["BS-CSMCI"], kind="reg")

Beispiel für ein Diagramm mit zu vielen Details :D

In [ ]:
import matplotlib.pyplot as plt
g = sns.jointplot(x=de20["BS-SFSH"], y=de20["BS-CSMCI"], scatter=False, kind="reg")
#g.annotate(stats.pearsonr)
# aktuelle Ersparnisse als Größe der Bubbles
# take only first month
de20s = de20[de20.index.month==1].copy()
# take every other year
de20s = de20s[::2]
# remove M01 from index
de20s.index = de20s.index.map(str).str.replace("-01-01 00:00:00", "")
sns.scatterplot(x=de20s["BS-SFSH"], y=de20s["BS-CSMCI"], s=de20s["BS-SV-PR"],
                hue=de20s.index, legend=True)
# Detailplot ohne Legende
sns.scatterplot(x=de20["BS-SFSH"], y=de20["BS-CSMCI"], s=de20["BS-SV-PR"],
                hue=de20.index, legend=False)
plt.gca().annotate("pearsonr=%0.2f p=%e"%stats.pearsonr(de20["BS-SFSH"], de20["BS-CSMCI"]), xy=(5, -25))

In [ ]:
import scipy.stats
scipy.stats.linregress(de20["BS-SFSH"], de20["BS-CSMCI"])

In [ ]:
!pip install geopandas
!wget https://github.com/datanizing/ix-jupyter-visualisierung/raw/main/europe.geo.json

In [ ]:
import geopandas
bl_geo = geopandas.read_file("europe.geo.json")
# die EU verwendet UK als Name, richtig ist aber GB
bl_geo.loc[bl_geo["iso_a2"] == "GB", "iso_a2"] = "UK"
bl_geo[["iso_a2", "geometry"]]

In [ ]:
# Now that Geopandas what is the "encoding" of your coordinates, you can perform any coordinate reprojection
#bl_geo = bl_geo.to_crs(epsg=3857)

In [ ]:
bl_geo[~bl_geo["iso_a2"].isin(["RU", "IS", "UA", "BY", "MD"])].plot(figsize=(10,10))

In [ ]:
hm = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
ghm = geopandas.GeoDataFrame(pd.merge(hm, bl_geo, left_on="country", right_on='iso_a2', how="outer"))

In [ ]:
ghm.plot(column=datetime(2020, 8, 1), legend=True, legend_kwds={'orientation': "horizontal"}, figsize=(10,10))

In [ ]:
ghm[~ghm["iso_a2"].isin(["RU", "IS", "UA", "BY", "MD"])].plot(column=datetime(2020, 8, 1), 
         legend=True, 
         legend_kwds={'orientation': "horizontal"}, 
         missing_kwds={
           "color": "lightgrey",
    },
         figsize=(10,10))

In [ ]:
toc = eurostat.get_toc_df()
toc

In [ ]:
import matplotlib.pyplot as plt
hm = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")].set_index("country")
columns = hm.columns
realnames = eurostat.get_dic('geo')
realnames["DE"] = "Germany"
#hm.index = hm.index.map(lambda i: realnames[i])
plt.figure(figsize=(24,24))
sns.heatmap(hm[sorted([c for c in hm.columns if isinstance(c, datetime) and c.year == 2020])].dropna(), cmap='viridis')

In [ ]:
hms = hm.reindex(["FI", "SE", "DK", "UK", "IE", "NL", "BE", "FR", "LU", "DE", "AT", "SI", "CZ", "HU", 
                  "SK", "PL", "LT", "LV", "EE", "HR", "IT", "ES", "PT", "MT", "EL", "CY", "RO", "BG", 
                  "RS", "MK", "ME", "AL", "TR"])
plt.figure(figsize=(24,24))
hms.index = hms.index.map(lambda i: realnames[i])
sns.heatmap(hms[sorted([c for c in hms.columns if isinstance(c, datetime) and c.year == 2020])].dropna(), cmap='viridis')

In [ ]:
pd.set_option('display.max_rows', 100)
hm[sorted([c for c in hm.columns if isinstance(c, datetime) and c.year == 2020])]

In [ ]:
hm.index